In [1]:
import numpy as np
import pandas as pd

Linear regression generally have the form of $Y_{i} = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + ...$ <br>
There are several ways to find the coefficients of the regression: <br>
1. Linear Algebra: $\hat{\theta} = (X^{T}X)^{-1}X^{T}Y$ (When X is invertible) <br>
2. Gradient Descent: In this case, we need to write out the loss function and try to minimize the loss. <br>
$\hspace{30mm}$ $F(x)$ = Loss Function = SE = $ \sum^{n}_{i=1} (Y_{i} - \hat{Y_{i}})^{2}$ <br>

In [2]:
class Linear_Regression():
    def __init__(self, alpha = 1e-10 , num_iter = 10000, early_stop = 1e-50, intercept = True, init_weight = None):
        
        
        """
            Some initializations, if neccesary
            
            attributes: 
                        alpha: Learning Rate, default 1e-10
                        num_iter: Number of Iterations to update coefficient with training data
                        early_stop: Constant control early_stop.
                        intercept: Bool, If we are going to fit a intercept, default True.
                        init_weight: Matrix (n x 1), input init_weight for testing.
                        
            
            TODO: 1. Initialize all variables needed.
        """
        
        self.model_name = 'Linear Regression'
        
        self.alpha = alpha
        self.num_iter = num_iter
        self.early_stop = early_stop
        self.intercept = intercept
        self.init_weight = init_weight  ### For testing correctness.
        
    
    def fit(self, X_train, y_train):
        """
            Save the datasets in our model, and perform gradient descent.
            
            Parameter:
                X_train: Matrix or 2-D array. Input feature matrix.
                Y_train: Matrix or 2-D array. Input target value.
                
                
                TODO: 2. If we are going to fit the intercept, add a col with all 1's to the first column. (hint: np.hstack, np.ones)
                      3. Initilaize our coef with uniform from [-1, 1] with the number of col in training set.
                      4. Call the gradient_descent function to train.
        """
        
        if self.intercept:
            ones_column = np.ones((X_train.shape[0], 1))
            self.X = np.hstack((ones_column, X_train))
        else:
            self.X = np.mat(X_train)       
        
        self.y = np.mat(y_train).T
        


        if not self.init_weight is None:
            self.coef = self.init_weight #### Please change this after you get the example right.
        else:
            self.coef = np.mat(np.random.uniform(-1,1,self.X.shape[1])).T
        self.gradient_descent()


    def gradient(self):
        """
            Helper function to calculate the gradient respect to coefficient.
            
            TODO: 5. Think about the matrix format of the gradient of the loss function.
           
            
        """

        #XT(Xw - y)
        self.grad_coef = -2 * self.X.T @ (self.y - self.X @ self.coef) 
        return self.grad_coef
        
        
        
    def gradient_descent(self):
        
        """
            Training function
            
            TODO: 6. Calculate the loss with current coefficients.
                  7. Update the temp_coef with learning rate and gradient.
                  8. Calculate the loss with temp_coef.
                  9. Implement the self adeptive learning rate. 
                      a. If current error is less than previous error, increase learning rate by a factor 1.3. 
                         And update coef, with temp_coef.
                      b. If previous error is less than current error, decrease learning rate by a factor of 0.9.
                         Don't update coef.
                  10. Add the loss to loss list we create.
        """

        self.loss = []
        
        for i in range(self.num_iter):
            self.gradient()

            previous_y_hat = self.X @ self.coef
                                                                                                    
            temp_coef = self.coef - self.alpha * self.grad_coef
            
            ones = np.mat(np.ones((1,self.X.shape[0])))  # Matrix with 1's (1 x n), help with calculate the sum of a mattrix. hint: Think about dot product.
            
            pre_error = ones @ np.square(self.y - previous_y_hat)
            
            current_error = ones @ np.square(self.y - self.X @ temp_coef)
            
            ### This is the early stop, don't modify fllowing three lines.
            if (abs(pre_error - current_error) < self.early_stop) | (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
                self.coef = temp_coef
                return self
            
            if current_error <= pre_error:
                self.coef = temp_coef
                self.alpha = self.alpha * 1.3
            else:
                self.alpha = self.alpha * 0.9
                
            self.loss.append(current_error[0,0])
            
            if i % 10000 == 0:
                print('Iteration: ' +  str(i))
                print('Coef: '+ str(self.coef))
                print('Loss: ' + str(current_error))            
        return self
    
    def ind_predict(self, x: list):
        """
            Predict the value based on its feature vector x.

            Parameter:
            x: Matrix, array or list. Input feature point.
            
            Return:
                result: prediction of given data point
        """
        
        """
            TODO: 11. Implement the prediction function
        """
        result = np.matrix(x).reshape((1, -1)) @ self.coef
        
        return result[0,0]
    
    def predict(self, X):
        """
            X is a matrix or 2-D numpy array, represnting testing instances. 
            Each testing instance is a feature vector. 
            
            Parameter:
            X: Matrix, array or list. Input feature point.
            
            Return:
                ret: prediction of given data matrix
        """
        
        """
            TODO: 12. Make sure add the 1's column like we did to add intercept.
                  13. Revise the following for-loop to call ind_predict to get predictions.

        """
        
        ret = []
        X = np.mat(X)
        if self.intercept:
            ones = np.ones((X.shape[0],1))
            X = np.hstack([ones, X])
        for x in X:
            ret.append(self.ind_predict(x))
        return ret
        
        

In [3]:
def min_max_normaliz(lst):
    """
    Helper function for normalize for faster training.
    """
    maximum = np.max(lst)
    minimum = np.min(lst)

    return (lst - minimum) / (maximum - minimum)

### We generate some easy data for testing. We should fit a line with, $Y = 30 * X + 20$

In [5]:
X = np.array(np.mat(np.arange(1, 1000, 5)).T)
y = np.array((30 * X)).flatten() +  20


#### Do NOT modify the following line, just run it when you are done.  You can also try different initialization, you will notice different coef at the end.

In [ ]:
clf = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight= np.mat([15,25]).T)
clf.fit(X,y)

####  As the number of iteration increase, you should notice the coeficient converges to [20, 30]. 
#### It maybe very slow update. Feel free to stop.

In [7]:
clf.coef

matrix([[15.36162795],
        [30.00698456]])

In [8]:
np.array(clf.predict(X))

array([   45.36861251,   195.40353534,   345.43845816,   495.47338098,
         645.5083038 ,   795.54322662,   945.57814944,  1095.61307226,
        1245.64799509,  1395.68291791,  1545.71784073,  1695.75276355,
        1845.78768637,  1995.82260919,  2145.85753201,  2295.89245483,
        2445.92737766,  2595.96230048,  2745.9972233 ,  2896.03214612,
        3046.06706894,  3196.10199176,  3346.13691458,  3496.1718374 ,
        3646.20676023,  3796.24168305,  3946.27660587,  4096.31152869,
        4246.34645151,  4396.38137433,  4546.41629715,  4696.45121998,
        4846.4861428 ,  4996.52106562,  5146.55598844,  5296.59091126,
        5446.62583408,  5596.6607569 ,  5746.69567972,  5896.73060255,
        6046.76552537,  6196.80044819,  6346.83537101,  6496.87029383,
        6646.90521665,  6796.94013947,  6946.9750623 ,  7097.00998512,
        7247.04490794,  7397.07983076,  7547.11475358,  7697.1496764 ,
        7847.18459922,  7997.21952204,  8147.25444487,  8297.28936769,
      

In [9]:
min_max_normaliz(X)

array([[0.        ],
       [0.00502513],
       [0.01005025],
       [0.01507538],
       [0.0201005 ],
       [0.02512563],
       [0.03015075],
       [0.03517588],
       [0.04020101],
       [0.04522613],
       [0.05025126],
       [0.05527638],
       [0.06030151],
       [0.06532663],
       [0.07035176],
       [0.07537688],
       [0.08040201],
       [0.08542714],
       [0.09045226],
       [0.09547739],
       [0.10050251],
       [0.10552764],
       [0.11055276],
       [0.11557789],
       [0.12060302],
       [0.12562814],
       [0.13065327],
       [0.13567839],
       [0.14070352],
       [0.14572864],
       [0.15075377],
       [0.15577889],
       [0.16080402],
       [0.16582915],
       [0.17085427],
       [0.1758794 ],
       [0.18090452],
       [0.18592965],
       [0.19095477],
       [0.1959799 ],
       [0.20100503],
       [0.20603015],
       [0.21105528],
       [0.2160804 ],
       [0.22110553],
       [0.22613065],
       [0.23115578],
       [0.236

#### Please try to normalize the X and fit again with normalized X. You should find something interesting. Also think about what you should do for predicting.

In [ ]:
clf = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight= np.mat([15,25]).T)
clf.fit(X,y)

##### You can also try this with the wine dataset we use in HW1. Try fit this function to that dataset with same features. If you look closely to the updates of coefficients. What do you find? This could be mentioned in your report. 

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(url_Wine, delimiter=';')
X = wine[['density','alcohol']]
y = wine.quality
print(X)

      density  alcohol
0     0.99780      9.4
1     0.99680      9.8
2     0.99700      9.8
3     0.99800      9.8
4     0.99780      9.4
...       ...      ...
1594  0.99490     10.5
1595  0.99512     11.2
1596  0.99574     11.0
1597  0.99547     10.2
1598  0.99549     11.0

[1599 rows x 2 columns]


In [13]:
lr = LinearRegression()
lr.fit(X,y)
## Squared Error with sklearn.
sum((lr.predict(X) - y)**2)

800.6676988774335

#### You will notice different coefficients, but the loss is very close to each other like 805. In your report, briefly discuss this problem.

In [14]:
clf = Linear_Regression(alpha = 1, num_iter = 100000, early_stop = 0)
clf.fit(X,y)

Iteration: 0
Coef: [[-0.01182732]
 [-0.66646641]
 [-0.99750631]]
Loss: [[5.74302532e+16]]
Iteration: 10000
Coef: [[0.86062003]
 [0.21462103]
 [0.43733026]]
Loss: [[816.43547388]]
Iteration: 20000
Coef: [[1.1174717 ]
 [0.47777818]
 [0.3873824 ]]
Loss: [[807.0053531]]
Iteration: 30000
Coef: [[1.20596419]
 [0.57153019]
 [0.37033354]]
Loss: [[805.8481003]]
Iteration: 40000
Coef: [[1.2354335 ]
 [0.60589467]
 [0.36425607]]
Loss: [[805.70605064]]
Iteration: 50000
Coef: [[1.24421838]
 [0.6194439 ]
 [0.3621063 ]]
Loss: [[805.68797566]]
Iteration: 60000
Coef: [[1.24576869]
 [0.62571401]
 [0.36137636]]
Loss: [[805.68516644]]
Iteration: 70000
Coef: [[1.24478091]
 [0.62943014]
 [0.36111156]]
Loss: [[805.68421567]]
Iteration: 80000
Coef: [[1.24290578]
 [0.63225327]
 [0.36102785]]
Loss: [[805.68349578]]
Iteration: 90000
Coef: [[1.24071818]
 [0.63476353]
 [0.36099837]]
Loss: [[805.68280498]]


In [15]:
sum((clf.predict(X) - y)**2)

805.6821176664736